In [1]:
import os
from urllib.request import urlopen
from urllib.request import urlretrieve
from urllib.parse import quote
from urllib.error import HTTPError
from bs4 import BeautifulSoup
from operator import eq

In [2]:
def getBs(url):
    try:
        html = urlopen(url)
    except HTTPError as e:
        return None
    try:
        bs = BeautifulSoup(html, 'html.parser')
    except AttributeError as e:
        return None
    return bs



In [3]:
## 크롤링 해야될 목록들
mainUrl = 'http://modernsweet.co.kr'

def get_category(url):
    bs = getBs(url)
    category = bs.ul.findAll('li',{'class':'xans-record-'})
    

    print('category list ')
    count = 1
    for item in category:
        print(count,')',item.a.get_text())
        count +=1
        
    while True:
        print('choice category (exit : q)')
        choice = input()
        for item in category:
            if eq(item.a.get_text(),choice):
                return item.a.attrs['href']
            if choice is 'q':
                return None
        print("can't find ",choice)
    

get_category(mainUrl)

category list 
1 ) BEST 50
2 ) ALL
3 ) OUTER
4 ) TOP
5 ) SHIRT
6 ) PANTS
7 ) BAG/SHOES
8 ) ACC
9 ) PERSONAL ORDER
choice category (exit : q)
ㅂ
can't find  ㅂ
choice category (exit : q)
q


In [4]:
## 다음 페이지 알아내기

def get_nextpage(url):
    bs = getBs(url)
    next_page = bs.find('img',{'alt':'다음 페이지'})
    next_page = next_page.parent.attrs['href']
    
    if eq(next_page,'#none'):
        return next_page
    
    andp = next_page.find('&') # 구분자 찾기
    next_page= next_page[andp:]
    return next_page

## 범위 벗어나면 #none
get_nextpage('http://modernsweet.co.kr/product/list.html?cate_no=70&page=5')

'&page=6'

In [5]:
## 페이지 내부에서 옷 목록 가져오기

def get_item(url):
    bs = getBs(url)
    item_list = bs.findAll('div',{'class':'box'})
    count = 1
    for item in item_list:
        try:
            print(count,') name : ',item.img.attrs['alt'])
            print('url :',item.a.attrs['href'])
            count+=1
        except AttributeError as e:
            print('no item')
    return item_list
item_list =  get_item('http://modernsweet.co.kr/product/list.html?cate_no=70')

1 ) name :  2019 여름 휴가 안내
url : /product/detail.html?product_no=1188&cate_no=70&display_group=1
2 ) name :  감탄 오버 반팔 티셔츠 20color
url : /product/detail.html?product_no=1187&cate_no=70&display_group=1
3 ) name :  마운틴 자수 하프 팬츠 4color
url : /product/detail.html?product_no=1186&cate_no=70&display_group=1
4 ) name :  230 슬림 일자 데님 팬츠(S~XL)
url : /product/detail.html?product_no=1185&cate_no=70&display_group=1
5 ) name :  아리조나 반팔 티 4color
url : /product/detail.html?product_no=1184&cate_no=70&display_group=1
6 ) name :  아레스 박스 오버 반팔티 3color
url : /product/detail.html?product_no=1183&cate_no=70&display_group=1
7 ) name :  SAROLEA 박스 나시티 3color
url : /product/detail.html?product_no=1182&cate_no=70&display_group=1
8 ) name :  타임 절개 카라 반팔티 8color
url : /product/detail.html?product_no=1181&cate_no=70&display_group=1
9 ) name :  코튼 워싱 카고 반바지 4color
url : /product/detail.html?product_no=1180&cate_no=70&display_group=1
10 ) name :  샌더스 반팔 니트 5color
url : /product/detail.html?product_no=1179&cate_no=70&d

In [6]:
### 현제 페이지에서 이미지 모두 가져오기
downloadDirectory = 'download'

def getDownloadLink(source):
    mainUrl = 'http://modernsweet.co.kr'
    source = quote(source,safe=':/')
    return mainUrl+source

def get_img(url,filename):
    bs = getBs(url)
    downpath = 'downdown/'+filename+'/'
    item_information = bs.find('div',{'id':'prdDetail'})
    img_list = item_information.findAll('img')
    count = 1
    for img in img_list:
        print(getDownloadLink(img.attrs['ec-data-src']))
        urlretrieve(getDownloadLink(img.attrs['ec-data-src']),downpath+str(count)+'.jpg')
        count+=1


In [7]:
def get_information(url):
    bs = getBs(url)
    info_list = bs.find('div',{'class':'cont'})
    info_list = info_list.findAll('span')
    count = 1
    text = ''
    for info in info_list:
        #print(count,':',info.get_text())
        #count+=1
        text += info.get_text()+'\n'
    return text

print(get_information('http://modernsweet.co.kr/product/detail.html?product_no=1180&cate_no=70&display_group=1'))


INFO
INFO
개인적으로도 강력 추천드리는 반바지 입니다:)
카고 형식으로 제작된 반바지에요.
탄탄한 면 소재이지만 답답하지 않고 짧은 기장감으로 제작되어
시원하고 캐쥬얼한 느낌으로 착용하기 좋으며
기본 무지 티셔츠와 매치하여도 
카고 디테일(포켓) 이 있어서 포인트를 주기 좋은 제품입니다:)
무엇보다 봄,가을 시즌 되면
맨투맨에 흰티 레이어드해서 입으면 정말 이쁜 제품이에요!
맨투맨에 흰티 레이어드해서 입으면 정말 이쁜 제품이에요!

무더운 여름에는 반팔티와 슬라브티 등을 매치하여
가볍고 편안하게 입으시고
FW 시즌되면 맨투맨과 티셔츠를 레이어드하여
남친룩을 연출하셔도 좋습니다:)
원단 자체가 워싱이 되어 제작된 제품이기 때문에
세탁 후에도 변형이나 뒤틀림이 전혀 없는 제품이라 매 시즌마다 꺼내어 입어주셔도 좋으며
허리 부분엔 뒷 밴딩으로 처리되어 있어 착용감도 굉장히 편하고
사이즈 또한 S,M,L,XL 까지 있어 남성분들 사이즈 선택에 폭도 넓습니다!
개인적으로는 컬러는 그레이, 블랙 제품이 어떤 상의와도 매치하기가 쉬워 추천드리며
남성분들 하나쯤 꼭 소장하시길 추천드리는 꿀템이라 생각합니다!
맨투맨과 연출된 코디컷은 곧 추가해서 보여드릴게요:)
COLOR
블랙, 오렌지, 화이트, 그레이
 
SIZE(cm)
S : 34(뒷밴딩) / 밑위 26 / 허벅지 32 / 밑단 28 / 총기장 39
M : 36(뒷밴딩) / 밑위 28 / 허벅지 33 / 밑단 29 / 총기장 40
L : 38(뒷밴딩) / 밑위 30 / 허벅지 34 / 밑단 30 / 총기장 41
XL : 40(뒷밴딩) / 밑위 32 / 허벅지 35 / 밑단 31 / 총기장 42
측정 위치에 따라 1-3CM 의 오차가 발생 할 수 있습니다.
FABRIC
면/스판
MODEL
174 CM / 63 KG
평소 28~29사이즈 착용 모델
S 사이즈 편안하게 착용하였습니다:)
DETAIL
비침 / 없음
두께감 / 보통
신축성 / 약간
그레이 / S 사이즈
DETAIL VIEW
화이트 / 그레이 / 오렌지 / 

In [8]:
def get_infofile(text,filename):

    path = 'downdown/'+filename+'/'
    f = open(path+filename+'.txt','w',-1, "utf-8")
    
    f.write(text)
    f.close()
    

In [9]:
def crawling():
    global mainUrl
    ## 메뉴 가져오기
    category = get_category(mainUrl)
    if category is None:
        return None
    ##메뉴 크롤링 시작
    print('start crawling')
    target = mainUrl+category
    count = 1
    while 1:
        print(count,' page')
        ##타겟 페이지 내부의 상품 추출
        item_list = get_item(target)
        for item in item_list:
            try:
                itemName = item.img.attrs['alt']
                itemUrl = item.a.attrs['href']
            except AttributeError as e:
                print('no item')
                break
                    
            os.makedirs('downdown/'+itemName)
            get_img(mainUrl+itemUrl,itemName)
            text = get_information(mainUrl+itemUrl)
            get_infofile(text,itemName)
            
            
        if eq(get_nextpage(target),'#none'):
            break
        else:
            target = target+get_nextpage(target)
        count += 1

In [10]:
crawling()

category list 
1 ) BEST 50
2 ) ALL
3 ) OUTER
4 ) TOP
5 ) SHIRT
6 ) PANTS
7 ) BAG/SHOES
8 ) ACC
9 ) PERSONAL ORDER
choice category (exit : q)
TOP
start crawling
1  page
1 ) name :  감탄 오버 반팔 티셔츠 20color
url : /product/detail.html?product_no=1187&cate_no=26&display_group=1
2 ) name :  아리조나 반팔 티 4color
url : /product/detail.html?product_no=1184&cate_no=26&display_group=1
3 ) name :  아레스 박스 오버 반팔티 3color
url : /product/detail.html?product_no=1183&cate_no=26&display_group=1
4 ) name :  SAROLEA 박스 나시티 3color
url : /product/detail.html?product_no=1182&cate_no=26&display_group=1
5 ) name :  타임 절개 카라 반팔티 8color
url : /product/detail.html?product_no=1181&cate_no=26&display_group=1
6 ) name :  샌더스 반팔 니트 5color
url : /product/detail.html?product_no=1179&cate_no=26&display_group=1
7 ) name :  네이쳐 린넨 긴팔 티 7color
url : /product/detail.html?product_no=1177&cate_no=26&display_group=1
8 ) name :  코튼 스트라이프 반팔 티 3color
url : /product/detail.html?product_no=1175&cate_no=26&display_group=1
9 ) name :  라메종 반팔

http://modernsweet.co.kr/web/upload/NNEditor/20190726/%EC%95%84%EB%A6%AC%EC%A1%B0%EB%82%98%20%EB%82%98%EC%97%BC%20%EB%B0%98%ED%8C%94%20%ED%8B%B0%20%2813%29_shop1_185843.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190726/%EC%95%84%EB%A6%AC%EC%A1%B0%EB%82%98%20%EB%82%98%EC%97%BC%20%EB%B0%98%ED%8C%94%20%ED%8B%B0%20%2814%29_shop1_185843.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190726/%EC%95%84%EB%A6%AC%EC%A1%B0%EB%82%98%20%EB%82%98%EC%97%BC%20%EB%B0%98%ED%8C%94%20%ED%8B%B0%20%2815%29_shop1_185843.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190726/%EC%95%84%EB%A6%AC%EC%A1%B0%EB%82%98%20%EB%82%98%EC%97%BC%20%EB%B0%98%ED%8C%94%20%ED%8B%B0%20%2816%29_shop1_185843.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190726/%EC%95%84%EB%A6%AC%EC%A1%B0%EB%82%98%20%EB%82%98%EC%97%BC%20%EB%B0%98%ED%8C%94%20%ED%8B%B0%20%2817%29_shop1_185843.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190726/%EC%95%84%EB%A6%AC%EC%A1%B0%EB%82%98%20%EB%82%98%EC%97%BC%20%EB%B0%98%ED%8C%94%20

http://modernsweet.co.kr/web/upload/NNEditor/20190726/%EC%95%84%EB%A0%88%EC%8A%A4%20%EB%B0%95%EC%8A%A4%20%EC%98%A4%EB%B2%84%20%EB%B0%98%ED%8C%94%ED%8B%B0%20%2824%29_shop1_185552.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190726/%EC%95%84%EB%A0%88%EC%8A%A4%20%EB%B0%95%EC%8A%A4%20%EC%98%A4%EB%B2%84%20%EB%B0%98%ED%8C%94%ED%8B%B0%20%2825%29_shop1_185552.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190726/%EC%95%84%EB%A0%88%EC%8A%A4%20%EB%B0%95%EC%8A%A4%20%EC%98%A4%EB%B2%84%20%EB%B0%98%ED%8C%94%ED%8B%B0%20%2826%29_shop1_185552.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190726/%EC%95%84%EB%A0%88%EC%8A%A4%20%EB%B0%95%EC%8A%A4%20%EC%98%A4%EB%B2%84%20%EB%B0%98%ED%8C%94%ED%8B%B0%20%2814%29_shop1_185557.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190726/%EC%95%84%EB%A0%88%EC%8A%A4%20%EB%B0%95%EC%8A%A4%20%EC%98%A4%EB%B2%84%20%EB%B0%98%ED%8C%94%ED%8B%B0%20%2815%29_shop1_185557.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190726/%EC%95%84%EB%A0%88%EC%8A%A4%20%EB%B0

http://modernsweet.co.kr/web/upload/NNEditor/20190724/%ED%83%80%EC%9E%84%20%EC%A0%88%EA%B0%9C%20%EC%B9%B4%EB%9D%BC%20%ED%8B%B0%EC%85%94%EC%B8%A0%20%2820%29_shop1_204218.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190724/%ED%83%80%EC%9E%84%20%EC%A0%88%EA%B0%9C%20%EC%B9%B4%EB%9D%BC%20%ED%8B%B0%EC%85%94%EC%B8%A0%20%2821%29_shop1_204219.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190724/%ED%83%80%EC%9E%84%20%EC%A0%88%EA%B0%9C%20%EC%B9%B4%EB%9D%BC%20%ED%8B%B0%EC%85%94%EC%B8%A0%20%2822%29_shop1_204219.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190724/%ED%83%80%EC%9E%84%20%EC%A0%88%EA%B0%9C%20%EC%B9%B4%EB%9D%BC%20%ED%8B%B0%EC%85%94%EC%B8%A0%20%281%29_shop1_204243.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190724/%ED%83%80%EC%9E%84%20%EC%A0%88%EA%B0%9C%20%EC%B9%B4%EB%9D%BC%20%ED%8B%B0%EC%85%94%EC%B8%A0%20%282%29_shop1_204243.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190724/%ED%83%80%EC%9E%84%20%EC%A0%88%EA%B0%9C%20%EC%B9%B4%EB%9D%BC%20%ED%8B%B0%EC%85%94%E

http://modernsweet.co.kr/web/upload/NNEditor/20190722/%EB%84%A4%EC%B8%84%EB%9F%B4%20%EB%A6%B0%EB%84%A8%20%EA%B8%B4%ED%8C%94%ED%8B%B0%20%2836%29_shop1_183937.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190722/%EB%84%A4%EC%B8%84%EB%9F%B4%20%EB%A6%B0%EB%84%A8%20%EA%B8%B4%ED%8C%94%ED%8B%B0%20%281%29_shop1_183946.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190722/%EB%84%A4%EC%B8%84%EB%9F%B4%20%EB%A6%B0%EB%84%A8%20%EA%B8%B4%ED%8C%94%ED%8B%B0%20%289%29_shop1_183946.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190722/%EB%84%A4%EC%B8%84%EB%9F%B4%20%EB%A6%B0%EB%84%A8%20%EA%B8%B4%ED%8C%94%ED%8B%B0%20%2810%29_shop1_183947.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190722/%EB%84%A4%EC%B8%84%EB%9F%B4%20%EB%A6%B0%EB%84%A8%20%EA%B8%B4%ED%8C%94%ED%8B%B0%20%2811%29_shop1_183947.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190722/%EB%84%A4%EC%B8%84%EB%9F%B4%20%EB%A6%B0%EB%84%A8%20%EA%B8%B4%ED%8C%94%ED%8B%B0%20%2812%29_shop1_183947.jpg
http://modernsweet.co.kr/web/upload/

...

http://modernsweet.co.kr/web/upload/NNEditor/20190717/%EC%8B%A4%EC%BC%93%20%EB%B0%98%ED%8C%94%20%ED%8B%B0%EC%85%94%EC%B8%A0%20%289%29_shop1_224010.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190717/%EC%8B%A4%EC%BC%93%20%EB%B0%98%ED%8C%94%20%ED%8B%B0%EC%85%94%EC%B8%A0%20%281%29_shop1_222604.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190717/%EC%8B%A4%EC%BC%93%20%EB%B0%98%ED%8C%94%20%ED%8B%B0%EC%85%94%EC%B8%A0%20%2817%29_shop1_222604.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190717/%EC%8B%A4%EC%BC%93%20%EB%B0%98%ED%8C%94%20%ED%8B%B0%EC%85%94%EC%B8%A0%20%2818%29_shop1_222604.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190717/%EC%8B%A4%EC%BC%93%20%EB%B0%98%ED%8C%94%20%ED%8B%B0%EC%85%94%EC%B8%A0%20%2819%29_shop1_222604.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190717/%EC%8B%A4%EC%BC%93%20%EB%B0%98%ED%8C%94%20%ED%8B%B0%EC%85%94%EC%B8%A0%20%2820%29_shop1_222605.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190717/%EC%8B%A4%EC%BC%93%20%EB%B0%98%ED%8C

http://modernsweet.co.kr/web/upload/NNEditor/20190713/%EC%8A%A4%ED%83%A0%EB%8B%A4%EB%93%9C%20%EC%8B%A4%EC%BC%93%20%EB%B0%98%ED%8C%94%20%ED%8B%B0%EC%85%94%EC%B8%A0%20%2832%29_shop1_181617.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190713/%EC%8A%A4%ED%83%A0%EB%8B%A4%EB%93%9C%EC%8B%A4%EC%BC%93%EB%B0%98%ED%8C%94%ED%8B%B0%20%281%29_shop1_181952.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190713/%EC%8A%A4%ED%83%A0%EB%8B%A4%EB%93%9C%EC%8B%A4%EC%BC%93%EB%B0%98%ED%8C%94%ED%8B%B0%20%283%29_shop1_181952.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190713/%EC%8A%A4%ED%83%A0%EB%8B%A4%EB%93%9C%EC%8B%A4%EC%BC%93%EB%B0%98%ED%8C%94%ED%8B%B0%20%284%29_shop1_181952.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190713/%EC%8A%A4%ED%83%A0%EB%8B%A4%EB%93%9C%EC%8B%A4%EC%BC%93%EB%B0%98%ED%8C%94%ED%8B%B0%20%282%29_shop1_182006.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190713/%EC%8A%A4%ED%83%A0%EB%8B%A4%EB%93%9C%EC%8B%A4%EC%BC%93%EB%B0%98%ED%8C%94%ED%8B%B0%20%285%29_shop1_18200

http://modernsweet.co.kr/web/upload/NNEditor/20190711/%EC%96%B4%EB%82%98%EB%8D%94%20%EB%B0%98%ED%8C%94%20%ED%8B%B0%EC%85%94%EC%B8%A0%20%2821%29_shop1_165758.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190711/%EC%96%B4%EB%82%98%EB%8D%94%20%EB%B0%98%ED%8C%94%20%ED%8B%B0%EC%85%94%EC%B8%A0%20%2822%29_shop1_165758.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190711/%EC%96%B4%EB%82%98%EB%8D%94%20%EB%B0%98%ED%8C%94%20%ED%8B%B0%EC%85%94%EC%B8%A0%20%2823%29_shop1_165758.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190711/%EC%96%B4%EB%82%98%EB%8D%94%20%EB%B0%98%ED%8C%94%20%ED%8B%B0%EC%85%94%EC%B8%A0%20%2824%29_shop1_165759.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190711/%EC%96%B4%EB%82%98%EB%8D%94%20%EB%B0%98%ED%8C%94%20%ED%8B%B0%EC%85%94%EC%B8%A0%20%2825%29_shop1_165759.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190711/%EC%96%B4%EB%82%98%EB%8D%94%20%EB%B0%98%ED%8C%94%20%ED%8B%B0%EC%85%94%EC%B8%A0%20%2826%29_shop1_165759.jpg
http://modernsweet.co.kr/web/uploa

http://modernsweet.co.kr/web/upload/NNEditor/20190710/%ED%81%AC%EB%9E%99-%EB%82%98%EC%97%BC-%EB%B0%98%ED%8C%94-%ED%8B%B0%EC%85%94%EC%B8%A0-%2839%29_shop1_192718.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190710/%ED%81%AC%EB%9E%99%20%EB%82%98%EC%97%BC%20%EB%B0%98%ED%8C%94%20%ED%8B%B0%EC%85%94%EC%B8%A0%20%2836%29_shop1_193550.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190710/%ED%81%AC%EB%9E%99%20%EB%82%98%EC%97%BC%20%EB%B0%98%ED%8C%94%20%ED%8B%B0%EC%85%94%EC%B8%A0%20%2837%29_shop1_193550.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190710/%ED%81%AC%EB%9E%99%20%EB%82%98%EC%97%BC%20%EB%B0%98%ED%8C%94%20%ED%8B%B0%EC%85%94%EC%B8%A0%20%2838%29_shop1_193550.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190710/%ED%81%AC%EB%9E%99%20%EB%82%98%EC%97%BC%20%EB%B0%98%ED%8C%94%20%ED%8B%B0%EC%85%94%EC%B8%A0%20%2839%29_shop1_193550.jpg
http://modernsweet.co.kr/web/upload/NNEditor/20190710/%ED%81%AC%EB%9E%99%20%EB%82%98%EC%97%BC%20%EB%B0%98%ED%8C%94%20%ED%8B%B0%EC%85%94%EC%B8%A

KeyboardInterrupt: 